In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer

In [2]:
train_users = pd.read_csv('../data/train_data.csv')

/Users/keonhoseo/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1243) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test_users = pd.read_csv('../data/test_data.csv')

In [4]:
train_users.fillna(-1, inplace=True)

In [5]:
y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id', 'most_used_device'], axis=1, inplace=True)
x_train = train_users.values

label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [22]:
x_test = test_users.drop(['id', 'most_used_device'], axis = 1).values

In [29]:
train_data = xgboost.DMatrix(x_train, encoded_y_train)

In [11]:
def ndcg5_score(preds, dtrain):
    labels = dtrain.get_label()
    top = []

    for i in range(preds.shape[0]):
        top.append(np.argsort(preds[i])[::-1][:5])

    mat = np.reshape(np.repeat(labels,np.shape(top)[1]) == np.array(top).ravel(),np.array(top).shape).astype(int)
    score = np.mean(np.sum(mat/np.log2(np.arange(2, mat.shape[1] + 2)),axis = 1))
    return 'ndcg5', score

In [30]:
param = {
    'max_depth': 8,
    'learning_rate': 1,
    'n_estimators': 5,
    'objective': 'multi:softprob',
    'num_class': 12,
    'gamma': 0,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'base_score': 0.5,
    'missing': None,
    'silent': True,
    'nthread': 4,
    'seed': 42
}

num_round = 10
xgboost.cv(param, train_data, num_boost_round=num_round, verbose_eval = True, metrics=['mlogloss'], feval=ndcg5_score)

[0]	train-mlogloss:1.19055+0.001299	train-ndcg5:0.83239+0.000176705	test-mlogloss:1.21417+0.00689788	test-ndcg5:0.827548+0.000859087
[1]	train-mlogloss:1.45821+0.00554659	train-ndcg5:0.755338+0.0024306	test-mlogloss:1.51094+0.00943521	test-ndcg5:0.744947+0.00345593
[2]	train-mlogloss:1.12708+0.0168378	train-ndcg5:0.836187+0.000897128	test-mlogloss:1.20004+0.00516024	test-ndcg5:0.822137+0.000537582
[3]	train-mlogloss:3.89355+0.0793912	train-ndcg5:0.775171+0.00761797	test-mlogloss:3.99319+0.13451	test-ndcg5:0.75786+0.00549442
[4]	train-mlogloss:1.51847+0.164038	train-ndcg5:0.836861+0.00552754	test-mlogloss:1.62851+0.15522	test-ndcg5:0.817384+0.0051638
[5]	train-mlogloss:1.94257+0.668361	train-ndcg5:0.830108+0.0141939	test-mlogloss:2.07341+0.642861	test-ndcg5:0.808339+0.0128339
[6]	train-mlogloss:1.57904+0.184578	train-ndcg5:0.841242+0.00181413	test-mlogloss:1.72138+0.159293	test-ndcg5:0.817419+0.00128098
[7]	train-mlogloss:1.63629+0.291983	train-ndcg5:0.842565+0.00361546	test-mlogloss:1.

,train-mlogloss-mean,train-mlogloss-std,train-ndcg5-mean,train-ndcg5-std,test-mlogloss-mean,test-mlogloss-std,test-ndcg5-mean,test-ndcg5-std
0,1.190554,0.001299,0.832390,0.000177,1.214168,0.006898,0.827548,0.000859
1,1.458212,0.005547,0.755338,0.002431,1.510940,0.009435,0.744947,0.003456
2,1.127079,0.016838,0.836187,0.000897,1.200042,0.005160,0.822137,0.000538
3,3.893554,0.079391,0.775171,0.007618,3.993191,0.134510,0.757860,0.005494
4,1.518466,0.164038,0.836861,0.005528,1.628515,0.155220,0.817384,0.005164
5,1.942568,0.668361,0.830108,0.014194,2.073409,0.642861,0.808339,0.012834
6,1.579043,0.184578,0.841242,0.001814,1.721384,0.159293,0.817419,0.001281
7,1.636293,0.291983,0.842565,0.003615,1.789549,0.264026,0.816488,0.002268
8,1.567674,0.292942,0.843837,0.004603,1.737137,0.272982,0.816177,0.003490
9,1.793499,0.248534,0.843362,0.001684,1.973616,0.223413,0.814036,0.000648


In [31]:
xgboost

<module 'xgboost' from '/Users/keonhoseo/opt/anaconda3/lib/python3.7/site-packages/xgboost/__init__.py'>